In [66]:
# Fusion ART Model for Professor Data using Word2Vec
# Python fusionART.py code by Dr. Budhitama Subagdja, modified for this dataset by Patrick Tjahjadi

# ----- Imported Libraries -----
from fusionART import *

from collections import defaultdict
from collections import Counter

import pandas as pd

import math

import random

from numpy import array
from numpy import argmax

from itertools import islice

from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize 
from nltk import ngrams
from nltk.corpus import stopwords

import gensim
from gensim.models import Word2Vec
from gensim.test.utils import datapath
import gensim.downloader as api
from gensim import corpora

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import csv

import wikipedia

import dill
# -------------------------------

data = pd.read_csv('SCSE ProfProfile.csv', encoding='ISO-8859-1')

In [3]:
# Load a session
dill.load_session('notebook_env_rep.db')

In [2]:
# Function to convert 1D lists into 2D, based on the number of elements per list.
def convert_to_2d(list_to_convert, list_per_entry): 
    it = iter(list_to_convert) 
    return [list(islice(it, i)) for i in list_per_entry] 

In [3]:
# Function to clean words from punctuation and remove capital case to standardise text tokens
def clean_text(word):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_123456789~'''
    no_punct = ""
    for char in word:
        if char not in punctuations:
            no_punct = no_punct + char
    return no_punct.lower()

In [4]:
# Function accepts a vector and normalise its values to a range between 0 and 1
def normalise_vector(vector):
    normalised_vector = [0] * len(vector)
    max_value = max(vector)
    min_value = min(vector)
    for index in range(0, len(vector)):
        normalised_vector[index] = (vector[index] - min_value) / (max_value - min_value)
    return normalised_vector

In [58]:
print(1 in (range(4,10) or range(5, 10)))

False


In [134]:
# Function to search for n-grams for n up to 6 and return the n-grammed text.
# Functions accepts list of n-grams to be formed, the text and the common n-grams within the text, respectively.
def search_for_ngrams(ngram_list, text_list):
    ngrammed_text = []
    for i in range(0, len(text_list)):
        bigram = text_list[i-1].lower()+"_"+text_list[i].lower()
        trigram = text_list[i-2].lower()+"_"+text_list[i-1]+"_"+text_list[i].lower()
        fourgram = text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        fivegram = text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        sixgram = text_list[i-5].lower()+"_"+text_list[i-4].lower()+"_"+text_list[i-3].lower()+"_"+text_list[i-2].lower()+"_"+text_list[i-1].lower()+"_"+text_list[i].lower()
        if bigram in ngram_list:
            ngrammed_text.append(bigram)
        if trigram in ngram_list:
            ngrammed_text.append(trigram)
        if fourgram in ngram_list:
            ngrammed_text.append(fourgram)
        if fivegram in ngram_list:
            ngrammed_text.append(fourgram)
        if sixgram in ngram_list:
            ngrammed_text.append(fourgram)
        ngrammed_text.append(text_list[i])
    return ngrammed_text


In [6]:
# Convert each attribute of the dataset to separate lists
name = []
group = []
university = []
research_interest = []
for idx, row in data.iterrows():
    name.append(row[0])
    group.append(row[1])
    university.append(row[2])
    research_interest.append(row[3])

# Problem: research_interest and university uses semicolons to denote multiple interests and universities
# Change their format to a list of universities and research interests of a person

university_preprocessed = []
for entry in university:
    uni = entry.split(";")
    university_preprocessed.append(uni)

research_interest_preprocessed = []
for entry in research_interest:
    interests = entry.split(";")
    research_interest_preprocessed.append(interests)    

    
# Break university_processed and research_interest_processed to 1D lists for one hot encoding
university_1d = []
for universities in university_preprocessed:
    for uni in universities:
        university_1d.append(uni)
        
research_interest_1d = []
for interests in research_interest_preprocessed:
    for interest in interests:
        research_interest_1d.append(interest)
        

In [7]:
# Determine the number of universities and research interests per professor

universities_per_person = []
for universities in university_preprocessed:
    universities_per_person.append(len(universities))
    

research_interest_per_person = []
for research_interests in research_interest_preprocessed:
    research_interest_per_person.append(len(research_interests))
    

In [8]:
# Research interest preprocessing, to consider each research keywords as a phrase
for index in range(0, len(research_interest_1d)):
    research_interest_1d[index] = research_interest_1d[index].replace(" ", "_")
    research_interest_1d[index] = research_interest_1d[index].lower()
    research_interest_1d[index] = research_interest_1d[index].replace(":", "")
    research_interest_1d[index] = research_interest_1d[index].replace(",", "")


In [9]:
# Create a list of unique names, groups, universities and research interests to define the model schema
# (I refrained from using sets because it would randomise the one-hot ordering)
unique_name = []
unique_group = []
unique_university = []
unique_research_interest = []
[unique_name.append(i) for i in name if i not in unique_name]
[unique_group.append(i) for i in group if i not in unique_group]
[unique_university.append(i) for i in university_1d if i not in unique_university]
[unique_research_interest.append(i) for i in research_interest_1d if i not in unique_research_interest]
pass


In [181]:
# Create a word2vec model that contains all research keywords
research_tokens = convert_to_2d(research_interest_1d, research_interest_per_person)
research_word2vec = Word2Vec(research_tokens, size = len(unique_research_interest), min_count = 0)


In [83]:
# Gather text tokens from Wikipedia articles for all research keywords 
wiki_text = []
for keyword in research_interest_1d:
    try:
        p = wikipedia.page(keyword)
        wiki_text.append(p.content.split())
        
    # Include exception handling if the Wikipedia article can not be found
    except wikipedia.exceptions.PageError:
        pass
    except wikipedia.DisambiguationError:
        pass



C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file C:\Users\patpa\Anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


In [164]:
# Remove common stop words from the retrieved Wikipedia articles
eng_stopwords = set(stopwords.words('english'))
for text in wiki_text:
    for word in text:
        if word in eng_stopwords:
            text.remove(word)

In [182]:
# Search for research keyword n-grams from Wikipedia articles and find most common n-grams

trigger = 0
for text in wiki_text:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)




In [183]:
# Create a list of common bigrams and trigrams from the Wikipedia articles
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))


In [184]:
# Search for research keyword n-grams from the Wikipedia articles and clean them
ngram_wiki_tokens = []

for text in wiki_text:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_wiki_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [185]:
# Train the word2vec model from the cleaned, n-grammed Wikipedia tokens
research_word2vec.build_vocab(ngram_wiki_tokens, update = True)
research_word2vec.train(ngram_wiki_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(3512795, 3627505)

In [186]:
# Open the custom corpus text file and tokenise them. Group them with 2500 words each
testtext = []
with open('it_text.txt','r', encoding="utf-8") as f:
    for line in f:
        for word in line.split():
            testtext.append(word)  
            

# Remove stopwords in the custom text corpus
for word in testtext:
    if word in eng_stopwords:
        testtext.remove(word)
        
test_tokens = convert_to_2d(testtext, [2500] * (math.ceil(len(testtext)/2500)))


In [187]:
# Search for research keyword n-grams from Wikipedia articles and find most common n-grams

trigger = 0
for text in test_tokens:
    string = " ".join(text)
    if (trigger == 0):
        bigram_counts = Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts = Counter(ngrams(string.split(), 3)).most_common(10)
        trigger += 1
    else:
        bigram_counts += Counter(ngrams(string.split(), 2)).most_common(10)
        trigram_counts += Counter(ngrams(string.split(), 3)).most_common(10)


In [188]:
# Create a list of common bigrams and trigrams from the custom corpus
common_ngrams = bigram_counts + trigram_counts
common_ngram_list = []
for each_ngram in common_ngrams:
    common_ngram_list.append("_".join(each_ngram[0]))

In [189]:
# Search for research keyword n-grams from the custom corpus and clean them
ngram_test_tokens = []

for text in test_tokens:
    cleaned_text = []
    for word in text:
        cleaned_text.append(clean_text(word))
    ngram_test_tokens.append(search_for_ngrams(unique_research_interest + common_ngram_list, cleaned_text))

In [190]:
# Train the word2vec model by including the cleaned, n-grammed custom corpus tokens
research_word2vec.build_vocab(ngram_test_tokens, update = True)
research_word2vec.train(ngram_test_tokens, total_examples = research_word2vec.corpus_count, epochs = research_word2vec.iter)

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  This is separate from the ipykernel package so we can avoid doing imports until


(148092, 162805)

In [175]:
# Example call to find similar analogies (e.g. computer_graphics + computer - graphics)
research_word2vec.wv.most_similar(positive=['computer_graphics', 'computer'], negative=['graphics'])

[('paperlike', 0.9202436208724976),
 ('computer_interaction', 0.9139674305915833),
 ('riskiest', 0.9085698127746582),
 ('computer_use', 0.9075106382369995),
 ('human_computer', 0.904493510723114),
 ('computer_simulations', 0.9035296440124512),
 ('human_computer_use', 0.8997400999069214),
 ('computer_program_runs', 0.8993540406227112),
 ('align', 0.8969988822937012),
 ('algorithmthe', 0.8969449400901794)]

In [176]:
# Example call to retrieve vector of a word
research_word2vec["ad_hoc_and_mobile_networks"]

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  


array([-5.17197594e-04,  6.34324271e-04, -7.28281098e-04,  6.48818852e-04,
       -2.09637554e-04, -7.38830131e-04,  1.58498980e-04, -5.51430894e-05,
        1.66240893e-03, -1.82720739e-03, -1.09217770e-03, -8.10529978e-04,
        1.44998694e-03, -6.20707578e-04,  4.29244188e-04, -5.61921159e-04,
        3.52414063e-04,  7.44485355e-04, -9.04765329e-04,  1.76641086e-04,
        9.78980330e-04,  1.42812112e-03, -1.79180468e-03,  1.26728439e-03,
       -7.63370132e-04, -5.37527885e-05,  9.83332051e-04,  1.53635294e-04,
       -1.73448212e-03, -1.38455222e-03, -8.20472778e-04,  1.77652703e-03,
       -1.65894791e-03,  1.12897076e-03, -8.13514402e-04, -8.45704926e-04,
        5.36590116e-04,  1.39961322e-03, -1.14090834e-03,  8.74884936e-05,
       -5.68231975e-04,  1.67956937e-03,  6.70620706e-04,  3.05859518e-04,
       -1.83971541e-03, -1.31656835e-03, -3.62012128e-04, -4.85016382e-04,
       -8.49089934e-04,  9.20358929e-04,  1.47236267e-03, -1.04589551e-03,
       -8.70875781e-04,  

In [177]:
# Calculate similarity score between two keywords
research_word2vec.wv.similarity("computer_graphics", "graphics")

0.9252301

In [191]:
# Find the top 5 most similar keywords for all research keywords
for keyword in sorted(unique_research_interest):
    print("Top 5 similar words from "+keyword+": ")
    print(research_word2vec.wv.most_similar(keyword, topn = 5))
    print()

Top 5 similar words from ad_hoc_and_mobile_networks: 
[('dictates', 0.17595764994621277), ('ducting', 0.17310819029808044), ('person_group_people', 0.1705676019191742), ('wellacknowledged', 0.1702735424041748), ('pointless', 0.1697530746459961)]

Top 5 similar words from agent_oriented_software_engineering: 
[('software_engineering', 0.9388728737831116), ('oriented_software_engineering', 0.9264748692512512), ('icse', 0.9205331206321716), ('conference_software_engineering', 0.9198015928268433), ('nrlarl', 0.9040243625640869)]

Top 5 similar words from algorithms: 
[('models', 0.9376203417778015), ('techniques', 0.9360089898109436), ('categorized', 0.9344889521598816), ('servicea', 0.9332371354103088), ('tensorbased', 0.9328782558441162)]

Top 5 similar words from algorithms_and_data_structure: 
[('visionrelated', 0.22544577717781067), ('automatic_in_between_frame_generation_from_hand_drawn_sketches', 0.20433060824871063), ('formantbased', 0.20345522463321686), ('enriches', 0.19430127739

[('systemtheoretic', 0.8366502523422241), ('abductionother', 0.835364043712616), ('analogical', 0.8239404559135437), ('designoriented', 0.8178521990776062), ('symbolically', 0.806858479976654)]

Top 5 similar words from computer_animation: 
[('computer_graphics', 0.9841339588165283), ('animation', 0.9772436022758484), ('field_computer_graphics', 0.9592769145965576), ('utilitarian', 0.9567065834999084), ('art', 0.9449078440666199)]

Top 5 similar words from computer_architecture: 
[('computer_simulation', 0.9601383209228516), ('computer_graphics_hardware', 0.9508353471755981), ('apollo', 0.944830596446991), ('computer_interfaces', 0.9448157548904419), ('computer_use', 0.9414820075035095)]

Top 5 similar words from computer_graphics: 
[('computer_animation', 0.9841339588165283), ('utilitarian', 0.9746754169464111), ('animation', 0.9635441303253174), ('field_computer_graphics', 0.9615151882171631), ('silicon_graphics', 0.957901120185852)]

Top 5 similar words from computer_networking: 
[(

[('surprise', 0.17723196744918823), ('bundles', 0.16417434811592102), ('multimedia_annotation', 0.15564969182014465), ('biologic', 0.15546751022338867), ('quantile', 0.15514616668224335)]

Top 5 similar words from entertainment_and_mobile_computing: 
[('siem', 0.19335849583148956), ('motionbased', 0.18909941613674164), ('use_primary_care', 0.18413443863391876), ('speech_processing_and_enhancement', 0.18156547844409943), ('reimers', 0.17456954717636108)]

Top 5 similar words from ethically_aligned_intelligent_decision_support: 
[('image_analysis_and_processing', 0.22387079894542694), ('cloud_computing_and_data_centre', 0.20167973637580872), ('data_stream_analytics', 0.16246318817138672), ('social_artificial_intelligence', 0.15601100027561188), ('servicing', 0.1539658159017563)]

Top 5 similar words from evolutionary_optimization: 
[('optimization_===', 0.9854927659034729), ('agents_===', 0.9833678007125854), ('multilinear', 0.9703681468963623), ('lightweight_formal_methods', 0.967371642


Top 5 similar words from model_checking: 
[('checking', 0.9847415685653687), ('stochastic', 0.9842180013656616), ('stmicroelectronics', 0.9803314805030823), ('iterative_algorithm', 0.9802049994468689), ('symbolic_model', 0.9801280498504639)]

Top 5 similar words from modelling_of_manipulative_interaction: 
[('multinode', 0.1808789223432541), ('user_preference_modeling_and_recommendation', 0.1795715093612671), ('scanreading', 0.1779608130455017), ('parallelizing', 0.16967427730560303), ('biologic', 0.16609764099121094)]

Top 5 similar words from modulation_and_multiple_access: 
[('materialsstructures', 0.1723584234714508), ('biochemical', 0.1667005717754364), ('safeguards', 0.166105717420578), ('lawton', 0.16283001005649567), ('siem', 0.15373094379901886)]

Top 5 similar words from multi_agent_system: 
[('multiagent_system', 0.8960285186767578), ('selforganized', 0.8905983567237854), ('structurebased', 0.8897073268890381), ('monolithic_system', 0.8889651298522949), ('ir_system', 0.8807

[('interferences', 0.20115484297275543), ('selectionbased', 0.1780746579170227), ('machinepressing', 0.1757025271654129), ('imager', 0.17492146790027618), ('dilated', 0.17420709133148193)]

Top 5 similar words from parallel_and_distributed_simulation: 
[('servicing', 0.15557369589805603), ('distributed_graph_computing', 0.14793987572193146), ('image_sequence_analysis_with_applications_to_digital_film_effects', 0.1405022144317627), ('exceedingly', 0.1403631567955017), ('benin', 0.13135063648223877)]

Top 5 similar words from pattern_recognition: 
[('pattern', 0.9767264723777771), ('projectionsome', 0.9580148458480835), ('learningbased', 0.9484095573425293), ('object_recognition', 0.9405184388160706), ('speech_recognition', 0.9369939565658569)]

Top 5 similar words from perceptual_visual_quality_gauging: 
[('ultra_wideband_radio', 0.21744130551815033), ('nonconformance', 0.1722431480884552), ('statevalue', 0.1492868959903717), ('ad_hoc_and_mobile_networks', 0.14669254422187805), ('sparsi


Top 5 similar words from software_agent: 
[('software_agents', 0.888448178768158), ('agentis', 0.8862788081169128), ('agentalso', 0.8821136951446533), ('software', 0.871325671672821), ('mlflex', 0.8557175397872925)]

Top 5 similar words from software_and_system_security: 
[('medical_image_processing_lesion_identification', 0.18826062977313995), ('embedded_signal_conversion_and_signal_processing', 0.17079725861549377), ('speech_processing_and_enhancement', 0.15933282673358917), ('multimedia_understanding_for_information_management', 0.15527503192424774), ('soft_computing', 0.13811957836151123)]

Top 5 similar words from software_engineering: 
[('icse', 0.9767967462539673), ('oriented_software_engineering', 0.9716972708702087), ('plenary', 0.968669056892395), ('biomedical_engineering', 0.9498666524887085), ('production—ian', 0.9442735314369202)]

Top 5 similar words from software_security: 
[('retested', 0.1785469651222229), ('computing_paradigm_computing', 0.17097802460193634), ('ecomm

[('a_growing_trend', 0.7942007184028625), ('plastics', 0.7622495889663696), ('communicator', 0.755090594291687), ('mobile_phones', 0.7434338331222534), ('crud', 0.7417265772819519)]



In [3]:
# One Hot Encoding for name, group and university
name_label_encoder = LabelEncoder()
uni_label_encoder = LabelEncoder()
group_label_encoder = LabelEncoder()
research_label_encoder = LabelEncoder()

onehot_encoder = OneHotEncoder(sparse=False)

name_integer_encoded = name_label_encoder.fit_transform(name)
name_integer_encoded = name_integer_encoded.reshape(len(name_integer_encoded), 1)
name_onehot = onehot_encoder.fit_transform(name_integer_encoded)

group_integer_encoded = group_label_encoder.fit_transform(group)
group_integer_encoded = group_integer_encoded.reshape(len(group_integer_encoded), 1)
group_onehot = onehot_encoder.fit_transform(group_integer_encoded)

university_integer_encoded = uni_label_encoder.fit_transform(array(university_1d))
university_integer_encoded = university_integer_encoded.reshape(len(university_integer_encoded), 1)
university_onehot = onehot_encoder.fit_transform(university_integer_encoded)


In [4]:
# Convert the numpy arrays to lists
name_onehotlist = name_onehot.tolist()
group_onehotlist = group_onehot.tolist()
university_onehotlist = university_onehot.tolist()



In [5]:
# Provide weight vectors for each research interest
research_interest_weights = []
for res_int in research_interest_1d:
    research_interest_weights.append(research_word2vec[res_int])

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [7]:
# Problem: each professor has multiple universities and research interests
# Convert these 1D arrays to 2D, to associate them to each professor

university_onehotlist_processed = []
research_interest_onehotlist_processed = []

university_onehotlist_processed = convert_to_2d(university_onehotlist, universities_per_person)
research_interest_processed = convert_to_2d(research_interest_weights, research_interest_per_person)

In [8]:
# Create an array that allows multiple universities and research interests per professor by summation
university_2d = []
for universities_per_professor in university_onehotlist_processed:
    university_2d.append([sum(x) for x in zip(*universities_per_professor)])
    
research_interest_2d = []
for research_interests_per_professor in research_interest_processed:
    research_interest_2d.append([sum(x) for x in zip(*research_interests_per_professor)])

# Since universities can be duplicate, change so that the maximum value of each university is 1
for universities_per_professor in university_2d:
    for index in range(0, len(universities_per_professor)):
        if universities_per_professor[index] > 1:
            universities_per_professor[index] = 1.0


In [10]:
# Normalise the values of all research keyword vectors to a range between 0 and 1
normalised_research_interest_2d = []
for research_vector in research_interest_2d:
    normalised_research_interest_2d.append(normalise_vector(research_vector))


In [11]:
# Example 2D research interest vector. This is the research keyword vector for Alexei Sourin
print(normalised_research_interest_2d[0])

[0.5139246405702792, 0.7242976226114285, 0.4955219755086069, 0.7364328750487741, 0.0, 0.5927072880885027, 0.6514367837467541, 0.7383984459525971, 0.7567196281420927, 0.7487676840321393, 0.6514027124781542, 0.688876213274125, 0.6760312278268745, 0.76186372171677, 0.7759539267705785, 0.6291905527907689, 0.6239928311612412, 0.55494008106371, 0.5877922884495624, 0.834729836419055, 0.8503740678036593, 0.7142380192993698, 0.7156082386046886, 0.5158776022258139, 0.6938219421490791, 0.7042739656242916, 0.4108688788966674, 0.815136670008132, 0.6120622006794351, 0.7832267508832511, 0.8920336043881857, 0.6443809024537057, 0.5187895567627583, 0.5943857916413339, 0.8118242282861456, 0.7477267457634411, 0.555517092017856, 0.6168428353727583, 0.6645471920322816, 0.5331118619828906, 0.483981380649962, 0.4535668260108613, 0.7408354126108996, 0.45095344430946727, 0.9031748120198935, 0.47993814921791367, 0.5774761768207071, 0.48215328568076016, 0.669436138289716, 0.639284626356913, 0.7472675712328738, 0.

In [12]:
# Define the schema for the FusionART model
model_schema = [{'name': data.columns[0], 'compl': False, 'attrib': sorted(unique_name)}, 
                {'name': data.columns[1], 'compl': False, 'attrib': sorted(unique_group)},
                {'name': data.columns[2], 'compl': False, 'attrib': sorted(unique_university)},
                {'name': data.columns[3], 'compl': True, 'attrib': sorted(unique_research_interest)}]

In [13]:
# Initialise the FusionART model
model = FusionART(schema = model_schema, beta = [1.0, 1.0, 1.0, 1.0], alpha = [0.1, 0.1, 0.1, 0.1], 
               gamma = [0.25, 0.25, 0.25, 0.25], rho = [1, 1, 1, 1])
model.F1Fields

[{'name': 'name',
  'compl': False,
  'attrib': ['AS Madhukumar',
   'Alexei Sourin',
   'Anupam Chattopadhyay',
   'Anwitaman Datta',
   'Arijit Khan',
   'Arvind Easwaran',
   'Bo An',
   'Cai Jianfei',
   'Cai Wentong',
   'Cham Tat Jen',
   'Chan Syin',
   'Chia Liang Tien Clement',
   'Chng Eng Siong',
   'Deepu Rajan',
   'Deng Ruilong',
   'Douglas Leslie Maskell',
   'Dusit Niyato',
   'Eric Cambria',
   'Gao Cong',
   'Goh Wooi Boon',
   'Guan Cuntai',
   'He Ying',
   'Huang Shell Ying',
   'Hui Siu Cheung',
   'Jagath C. Rajapakse',
   'Ke Yiping Kelly',
   'Kong Wai Kin Adams',
   'Kwoh Chee Keong',
   'Lam Kwok Yan',
   'Lam Siew Kei',
   'Lau Chiew Tong',
   'Lee Bu Sung Francis',
   'Li Fang Flora',
   'Li Mo',
   'Li Yi',
   'Liang Qianhui Althea',
   'Lin Feng',
   'Lin Guosheng',
   'Lin Shang Wei',
   'Lin Weisi',
   'Liu Weichen',
   'Liu Yang',
   'Loke Yuan Ren',
   'Long Cheng',
   'Loy Chen Change (Cavan)',
   'Lu Shijian',
   'Luo Jun',
   'Mahardhika Pratama',

In [14]:
# Store the data to the Fusion ART model

for i in range(0, len(name)):
    model.updateF1bySchema([{'name': data.columns[0], 'val': name_onehotlist[i]}, 
                            {'name': data.columns[1], 'val': group_onehotlist[i]}, 
                            {'name': data.columns[2], 'val': university_2d[i]}, 
                            {'name': data.columns[3], 'val': normalised_research_interest_2d[i]}])

    
    print("resonance search: ")
    J = model.resSearch()
    print("selected ", J)
    if model.uncommitted(J):
        print ('uncommitted')

    model.autoLearn(J)
    model.doReadoutAllFields(J)
                    
    
model.displayNetwork()

resonance search: 
selected  0
uncommitted
resonance search: 
selected  1
uncommitted
resonance search: 
selected  2
uncommitted
resonance search: 
selected  3
uncommitted
resonance search: 
selected  4
uncommitted
resonance search: 
selected  5
uncommitted
resonance search: 
selected  6
uncommitted
resonance search: 
selected  7
uncommitted
resonance search: 
selected  8
uncommitted
resonance search: 
selected  9
uncommitted
resonance search: 
selected  10
uncommitted
resonance search: 
selected  11
uncommitted
resonance search: 
selected  12
uncommitted
resonance search: 
selected  13
uncommitted
resonance search: 
selected  14
uncommitted
resonance search: 
selected  15
uncommitted
resonance search: 
selected  16
uncommitted
resonance search: 
selected  17
uncommitted
resonance search: 
selected  18
uncommitted
resonance search: 
selected  19
uncommitted
resonance search: 
selected  20
uncommitted
resonance search: 
selected  21
uncommitted
resonance search: 
selected  22
uncommitte

Code: 28 {'F2': 0, 'weights': [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.64357

In [6]:
dill.dump_session('notebook_env_rep.db')

In [9]:
# Perform the research keywords query here

def query_by_research_with_noise(query_research, output_file, noise_limit):
    
    query_research = query_research.split(";")
    output_file.write("Research Keywords: ")
    output_file.write(', '.join(query_research)+"\n")

    # Preprocess the research keyword queries to the format used for word embedding (phrases)
    for index in range(0, len(query_research)):
        query_research[index] = query_research[index].replace(" ", "_")
        query_research[index] = query_research[index].lower()
        query_research[index] = query_research[index].replace(":", "")
        query_research[index] = query_research[index].replace(",", "")

    # Convert all the research keywords into input vectors
    for research_index in range(0, len(query_research)):
        query_research[research_index] = research_word2vec[query_research[research_index]].tolist()


    # Remove keywords that could not be converted to input vectors
    index = 0
    while (index < len(query_research)):
        if type(query_research[index]) != list:
            del query_research[index]
        else:
            index += 1
        
    # Combine all the research keywords inputs into one vector that contains all research keywords
    research_vector = [sum(research_ints) for research_ints in zip(*query_research)]
    
    normalised_research_vector = normalise_vector(research_vector)
    

    # With a defined probability, flip the binary research keyword vectors due to noise
    for index in range(0, len(research_vector)):
        noise_value = random.random()
        if (noise_value < noise_limit):
            normalised_research_vector[index] = 1 - normalised_research_vector[index]
            
    # Define the complement-coded research vector
    compl_vector = []
    for bit in normalised_research_vector:
        compl_vector.append(1 - bit)
    

    model.setParam('gamma', [0,0,0,1])
    model.setParam('rho', [0,0,0,0])
    model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector, 'vcompl': compl_vector}], refresh = False)
    # model.updateF1bySchema([{'name': data.columns[3], 'val': normalised_research_vector}])
    model.compChoice()

    # Find the nodes that have the highest F2 value
    maxF2value = model.codes[0]['F2']
    maxF2indexes = []

    for i in range(0, len(model.codes)-1):
        F2value = model.codes[i]['F2']
        if (F2value > maxF2value):
            maxF2value = F2value
            maxF2indexes = [i]
        elif (F2value == maxF2value):
            maxF2indexes.append(i)

    # Retrieve the data of the node(s) that have the highest F2 value
    output_file.write("Professor that follows these research keywords:")    
    for node in maxF2indexes:
        name_vector = model.codes[node]['weights'][0]
        for index in range(0, len(name_vector)):
            # Iterate through the vector and find the indexes that has a value of 1
            if (name_vector[index] == 1):
                # Retrieve the data from the model's 'attrib' attribute in F1 
                retrieved_name = model.F1Fields[0]['attrib'][index]
                output_file.write(retrieved_name)
    
    output_file.write("\n\n")
    return(retrieved_name)


In [74]:
# Research keywords query for 0% noise
output_file = open("output_zero_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0)
        if (retrieved_name == batch_name):
            correct_match += 1
        break
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()


[0.49537055912022454, 0.927142673688341, 0.5361519131375191, 0.5336055674013777, 0.3501897515381416, 0.31836508423879, 0.3498349261682984, 0.4073620839066943, 0.5922694535645383, 0.6513677682260025, 0.6078025863573863, 0.42106590468639876, 0.8027896796905829, 0.6181485808494006, 0.38348948386836756, 0.7326754639102909, 0.6545912491274466, 0.4570107803550435, 0.26396504554104183, 0.4935568512031855, 0.5327718930954388, 0.6880727494364531, 0.5121772587233607, 0.5931946811583624, 0.4380220348785011, 0.46048677902003354, 0.7362960078110512, 0.6459839440804764, 0.8734011505632775, 0.787423336238077, 0.6702241685153042, 0.537308770238589, 0.7736056940553226, 0.7900905548117869, 0.8315707470714048, 0.8033600425339887, 0.7374819365974294, 0.5444085024770421, 0.4322989776005031, 0.6026465771311253, 0.4251038254002212, 0.7900890445249591, 0.7577414387042508, 0.7921640273314098, 0.7771272629246805, 0.5903125240801668, 0.6579860349760096, 0.3773457664674701, 0.5444790614707197, 0.4397789489908239,

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [48]:
# Research keywords query for 10% noise
output_file = open("output_ten_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [49]:
# Research keywords query for 20% noise
output_file = open("output_twenty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.2)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [50]:
# Research keywords query for 30% noise
output_file = open("output_thirty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.3)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [51]:
# Research keywords query for 40% noise
output_file = open("output_forty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.4)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [52]:
# Research keywords query for 50% noise
output_file = open("output_fifty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.5)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [53]:
# Research keywords query for 60% noise
output_file = open("output_sixty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.6)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [54]:
# Research keywords query for 70% noise
output_file = open("output_seventy_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.7)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [55]:
# Research keywords query for 80% noise
output_file = open("output_eighty_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.8)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [56]:
# Research keywords query for 90% noise
output_file = open("output_ninety_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 0.9)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [57]:
# Research keywords query for 100% noise
output_file = open("output_hundred_noise.txt", "w+")
correct_match = 0

# Use a batch file as input for the research keywords query
with open('research_query.bat', 'r') as f:
    reader = csv.reader(f, delimiter = '\t')
    for batch_name, keywords in reader:
        retrieved_name = query_by_research_with_noise(keywords, output_file, 1)
        if (retrieved_name == batch_name):
            correct_match += 1
            
query_accuracy = (correct_match / len(name)) * 100
output_file.write("\nQuery Accuracy: "+str(query_accuracy)+"%")
output_file.close()

C:\Users\patpa\Anaconda3\lib\site-packages\ipykernel_launcher.py:18: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
